In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Dense, Flatten, Dropout, GlobalAveragePooling1D
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelBinarizer

# Load the preprocessed data
X_train = np.loadtxt("../Datasets/CAPP Dataset/SubjectIndependent50PercentOverlap/X_train.txt")
y_train = np.loadtxt("../Datasets/CAPP Dataset/SubjectIndependent50PercentOverlap/y_train.txt")
X_test = np.loadtxt("../Datasets/CAPP Dataset/SubjectIndependent50PercentOverlap/X_test.txt")
y_test = np.loadtxt("../Datasets/CAPP Dataset/SubjectIndependent50PercentOverlap/y_test.txt")

# Reshape the input data for Conv1D layers
n_timestep = 100
n_features = 9
X_train = X_train.reshape(X_train.shape[0], n_timestep, n_features)
X_test = X_test.reshape(X_test.shape[0], n_timestep, n_features)

# One-hot encode the labels
lb = LabelBinarizer()
y_train = lb.fit_transform(y_train)
y_test = lb.transform(y_test)

def create_model(n_timesteps, n_features, n_outputs):
    model = Sequential([
        Conv1D(filters=64, kernel_size=3, activation="relu", input_shape=(n_timesteps, n_features)),
        Conv1D(filters=64, kernel_size=3, activation="relu"),
        GlobalAveragePooling1D(),
        Dense(128, activation="relu"),
        Dropout(0.5),
        Dense(64, activation="relu"),
        Dense(n_outputs, activation="softmax")
    ])
    return model

n_outputs = y_train.shape[1]
conv_mlp = create_model(n_timestep, n_features, n_outputs)
conv_mlp.compile(optimizer=Adam(learning_rate=0.001), loss="categorical_crossentropy", metrics=["accuracy"])

# Train the model
history = conv_mlp.fit(X_train, y_train, epochs=500, batch_size=512, validation_split=0.2, verbose=1)

# Evaluate the model
y_pred = conv_mlp.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(y_test, axis=1)

# Print classification report
print(classification_report(y_true_classes, y_pred_classes, digits=5))

/home/denuvo-drm/miniconda3/envs/CompositeADLRecognition/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/500
156/156 ━━━━━━━━━━━━━━━━━━━━ 4s 15ms/step - accuracy: 0.2590 - loss: 2.3499 - val_accuracy: 0.6036 - val_loss: 1.2346
Epoch 2/500
156/156 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6355 - loss: 1.1177 - val_accuracy: 0.6735 - val_loss: 1.0234
Epoch 3/500
156/156 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7203 - loss: 0.8588 - val_accuracy: 0.7043 - val_loss: 0.9290
Epoch 4/500
156/156 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7581 - loss: 0.7475 - val_accuracy: 0.7162 - val_loss: 0.9354
Epoch 5/500
156/156 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7790 - loss: 0.6776 - val_accuracy: 0.7164 - val_loss: 0.9329
Epoch 6/500
156/156 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7944 - loss: 0.6294 - val_accuracy: 0.7443 - val_loss: 0.8891
Epoch 7/500
156/156 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8080 - loss: 0.5873 - val_accuracy: 0.7228 - val_loss: 0.9326
Epoch 8/500
156/156 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8205 - loss: 0.5510 - val_acc

In [3]:
# Convert the model to TensorFlow Lite format
converter = tf.lite.TFLiteConverter.from_keras_model(conv_mlp)
tflite_model = converter.convert()

# Save the TFLite model
with open("SubjectDependent/GeneratedTFLiteFilesAndOGModels/conv_mlp_model.tflite", "wb") as f:
    f.write(tflite_model)

print("TensorFlow Lite model has been saved as 'conv_mlp_model.tflite'")

# Optional: Save the Keras model
conv_mlp.save("GeneratedTFLiteFilesAndOGModels/conv_mlp_model.h5")
print("Keras model has been saved as 'conv_mlp_model.h5'")

# If you still need to generate a C header file
try:
    from everywhereml.code_generators.tensorflow import convert_model

    c_header = convert_model(conv_mlp, X_test, y_test, model_name='conv_mlp_model')

    with open("SubjectDependent/GeneratedHeaderFiles/conv_mlp_model.h", "w") as file:
        file.write(c_header)

    print("C header file has been saved as GeneratedHeaderFiles/conv_mlp_model.h'")
except ImportError:
    print("everywhereml library not found. Skipping C header file generation.")


INFO:tensorflow:Assets written to: /tmp/tmpo7pgwsgd/assets


INFO:tensorflow:Assets written to: /tmp/tmpo7pgwsgd/assets


Saved artifact at '/tmp/tmpo7pgwsgd'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 100, 9), dtype=tf.float32, name='keras_tensor_8')
Output Type:
  TensorSpec(shape=(None, 21), dtype=tf.float32, name=None)
Captures:
  130758116849296: TensorSpec(shape=(), dtype=tf.resource, name=None)
  130758116848944: TensorSpec(shape=(), dtype=tf.resource, name=None)
  130758116849472: TensorSpec(shape=(), dtype=tf.resource, name=None)
  130758112317104: TensorSpec(shape=(), dtype=tf.resource, name=None)
  130758116851056: TensorSpec(shape=(), dtype=tf.resource, name=None)
  130758112318864: TensorSpec(shape=(), dtype=tf.resource, name=None)
  130758112315696: TensorSpec(shape=(), dtype=tf.resource, name=None)
  130758112320800: TensorSpec(shape=(), dtype=tf.resource, name=None)
  130758112314640: TensorSpec(shape=(), dtype=tf.resource, name=None)
  130758112322384: TensorSpec(shape=(), dtype=tf.resource, name=None)


W0000 00:00:1727397570.143308  116468 tf_tfl_flatbuffer_helpers.cc:392] Ignored output_format.
W0000 00:00:1727397570.143321  116468 tf_tfl_flatbuffer_helpers.cc:395] Ignored drop_control_dependency.
2024-09-27 06:39:30.143528: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmpo7pgwsgd
2024-09-27 06:39:30.143873: I tensorflow/cc/saved_model/reader.cc:52] Reading meta graph with tags { serve }
2024-09-27 06:39:30.143881: I tensorflow/cc/saved_model/reader.cc:147] Reading SavedModel debug info (if present) from: /tmp/tmpo7pgwsgd
2024-09-27 06:39:30.146992: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:388] MLIR V1 optimization pass is not enabled
2024-09-27 06:39:30.147631: I tensorflow/cc/saved_model/loader.cc:236] Restoring SavedModel bundle.
2024-09-27 06:39:30.166967: I tensorflow/cc/saved_model/loader.cc:220] Running initialization op on SavedModel bundle at path: /tmp/tmpo7pgwsgd
2024-09-27 06:39:30.172989: I tensorflow/cc/saved_model/loader.cc

TensorFlow Lite model has been saved as 'conv_mlp_model.tflite'
Keras model has been saved as 'conv_mlp_model.h5'
INFO:tensorflow:Assets written to: /tmp/tmp7zdj70iq/assets


INFO:tensorflow:Assets written to: /tmp/tmp7zdj70iq/assets


Saved artifact at '/tmp/tmp7zdj70iq'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 100, 9), dtype=tf.float32, name='keras_tensor_8')
Output Type:
  TensorSpec(shape=(None, 21), dtype=tf.float32, name=None)
Captures:
  130758116849296: TensorSpec(shape=(), dtype=tf.resource, name=None)
  130758116848944: TensorSpec(shape=(), dtype=tf.resource, name=None)
  130758116849472: TensorSpec(shape=(), dtype=tf.resource, name=None)
  130758112317104: TensorSpec(shape=(), dtype=tf.resource, name=None)
  130758116851056: TensorSpec(shape=(), dtype=tf.resource, name=None)
  130758112318864: TensorSpec(shape=(), dtype=tf.resource, name=None)
  130758112315696: TensorSpec(shape=(), dtype=tf.resource, name=None)
  130758112320800: TensorSpec(shape=(), dtype=tf.resource, name=None)
  130758112314640: TensorSpec(shape=(), dtype=tf.resource, name=None)
  130758112322384: TensorSpec(shape=(), dtype=tf.resource, name=None)


W0000 00:00:1727397570.489396  116468 tf_tfl_flatbuffer_helpers.cc:392] Ignored output_format.
W0000 00:00:1727397570.489408  116468 tf_tfl_flatbuffer_helpers.cc:395] Ignored drop_control_dependency.
2024-09-27 06:39:30.489521: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmp7zdj70iq
2024-09-27 06:39:30.489873: I tensorflow/cc/saved_model/reader.cc:52] Reading meta graph with tags { serve }
2024-09-27 06:39:30.489883: I tensorflow/cc/saved_model/reader.cc:147] Reading SavedModel debug info (if present) from: /tmp/tmp7zdj70iq
2024-09-27 06:39:30.493250: I tensorflow/cc/saved_model/loader.cc:236] Restoring SavedModel bundle.
2024-09-27 06:39:30.512298: I tensorflow/cc/saved_model/loader.cc:220] Running initialization op on SavedModel bundle at path: /tmp/tmp7zdj70iq
2024-09-27 06:39:30.518075: I tensorflow/cc/saved_model/loader.cc:462] SavedModel load for tags { serve }; Status: success: OK. Took 28556 microseconds.


C header file has been saved as GeneratedHeaderFiles/conv_mlp_model.h'


In [4]:
# Testing out the TFLite model when optimized for normal edge devices
from sklearn.metrics import accuracy_score

# Loading the test data
X_test = np.loadtxt("../Datasets/CAPP Dataset/SubjectIndependent50PercentOverlap/X_test.txt")
y_test = np.loadtxt("../Datasets/CAPP Dataset/SubjectIndependent50PercentOverlap/y_test.txt")

# Reshaping the input data
n_timestep = 100
n_features = 9
X_test = X_test.reshape(X_test.shape[0], n_timestep, n_features)

# Doing One-hot encode the labels
lb = LabelBinarizer()
y_test = lb.fit_transform(y_test)

# Loading the TFLite model
interpreter = tf.lite.Interpreter(model_path="SubjectDependent/GeneratedTFLiteFilesAndOGModels/conv_mlp_model.tflite")
interpreter.allocate_tensors()

# Allocating input and output tensors
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Test the model on random input data
input_shape = input_details[0]["shape"]
input_data = X_test

# Now running predictions on every test sample
y_pred_tflite = []
for i in range(len(input_data)):
    input_tensor = np.array(input_data[i], dtype=np.float32)
    input_tensor = np.expand_dims(input_tensor, axis=0)
    interpreter.set_tensor(input_details[0]["index"], input_tensor)
    interpreter.invoke()
    output_data = interpreter.get_tensor(output_details[0]["index"])
    y_pred_tflite.append(output_data)

y_pred_tflite = np.vstack(y_pred_tflite)

# Converting predictions to class labels
y_pred_classes_tflite = np.argmax(y_pred_tflite, axis=1)
y_true_classes = np.argmax(y_test, axis=1)

# Calculating the accuracy
accuracy_tflite = accuracy_score(y_true_classes, y_pred_classes_tflite)
print(f"TFLite Model Accuracy: {accuracy_tflite:.5f}")

print("\nClassification Report:")
print(classification_report(y_true_classes, y_pred_classes_tflite, digits=5))

try:
    original_model = tf.keras.models.load_model("GeneratedTFLiteFilesAndOGModels/conv_mlp_model.h5")
    y_pred_original = original_model.predict(X_test)
    y_pred_classes_original = np.argmax(y_pred_original, axis=1)
    accuracy_original = accuracy_score(y_true_classes, y_pred_classes_original)
    print(f"\nOriginal Keras Model Accuracy: {accuracy_original:.5f}")
    
    prediction_match = np.mean(y_pred_classes_tflite == y_pred_classes_original)
    print(f"\nPrediction Match between TFLite and Original model: {prediction_match:.5f}")
except:
    print("\nError Check your path")

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


TFLite Model Accuracy: 0.74664

Classification Report:
              precision    recall  f1-score   support

           0    0.89556   0.86947   0.88232       927
           1    0.98230   0.95793   0.96996       927
           2    0.86013   0.89177   0.87566       924
           3    0.96692   0.86659   0.91401       877
           4    0.82784   0.71670   0.76827       946
           5    0.61398   0.68191   0.64617       940
           6    0.83825   0.93326   0.88321       944
           7    0.77551   0.88000   0.82446       950
           8    0.85843   0.29968   0.44427       951
           9    0.67094   0.33873   0.45018       927
          10    0.85984   0.72390   0.78604      1322
          11    0.58785   0.96099   0.72948       564
          12    0.88465   0.96999   0.92536       933
          13    0.60838   0.90150   0.72649       934
          14    0.62705   0.69043   0.65722       940
          15    0.61419   0.76509   0.68138       928
          16    0.98563   